In [1]:
import os
import sys
import pickle
import csv
import numpy as np

In [2]:
with open('../../temp/norm_coef_pascal.pkl', 'rb') as fin:
    norm_coef_pascal = pickle.load(fin)

In [3]:
with open('../../temp/norm_coef_coco.pkl', 'rb') as fin:
    norm_coef_coco = pickle.load(fin)

In [4]:
with open('../../temp/norm_coef_ws.pkl', 'rb') as fin:
    norm_coef_ws = pickle.load(fin)

In [5]:
norm_coef_coco

{'/m/012xff': {'mean_oi': 0.4390718101996786,
  'std_oi': 0.29126858122679394,
  'mean_coco': 0.9284473806328127,
  'std_coco': 0.12175547390481509},
 '/m/0l515': {'mean_oi': 0.2408217621624418,
  'std_oi': 0.2407899885307404,
  'mean_coco': 0.9436722959429897,
  'std_coco': 0.0919032716009391},
 '/m/04dr76w': {'mean_oi': 0.4616209371020139,
  'std_oi': 0.2833081832956493,
  'mean_coco': 0.9430647492408752,
  'std_coco': 0.13013429546697713},
 '/m/04_sv': {'mean_oi': 0.5362437685392248,
  'std_oi': 0.3382215695202557,
  'mean_coco': 0.9425050746210496,
  'std_coco': 0.10346537314519029},
 '/m/0bt9lr': {'mean_oi': 0.6202079311835591,
  'std_oi': 0.29043030326198305,
  'mean_coco': 0.9604561309401806,
  'std_coco': 0.10918921346254076},
 '/m/0cmx8': {'mean_oi': 0.4390718101996786,
  'std_oi': 0.29126858122679394,
  'mean_coco': 0.9284473806328127,
  'std_coco': 0.12175547390481509},
 '/m/04bcr3': {'mean_oi': 0.34127212477067265,
  'std_oi': 0.2903532916647331,
  'mean_coco': 0.7121611285

## Concatenating the results 

In [5]:
with open('../../../tfobjdetect/results/det_results_m9_jpg_oi_1.pkl', 'rb') as fin:
    det_results_oi_seed = pickle.load(fin)

with open('../../../tfobjdetect/results/det_results_m9_jpg_coco_1.pkl', 'rb') as fin:
    det_results_coco_seed = pickle.load(fin)

with open('../../results/det_results_ram_m9_jpg_1', 'rb') as fin:
    det_results_pascal_seed = pickle.load(fin)

with open('../../../wsod/results/det_results_dpl_027.pkl', 'rb') as fin:
    det_results_ws_seed = pickle.load(fin)
    


In [6]:
with open('../../../wsod/metadata/ont_m18/mapping.pkl', 'rb') as fin:
    mid2ont, syn2mid, single_mids, mid2syn, class2ont, ont2name, class_names = pickle.load(fin)  

In [7]:
label_set = set()
for imgid, det_list in det_results_oi_seed.items():
    for det in det_list:
        if det['label'] in mid2syn:
            label_set.add(det['label'])
            det['label'] = mid2syn[det['label']]

In [8]:
label_set

set()

In [9]:
det_results_concat = {key:[] for key in det_results_oi_seed}
for imgid in det_results_oi_seed:
    for det in det_results_oi_seed[imgid]:
        det_results_concat[imgid].append({
            'label': det['label'],
            'score': det['score'],
            'bbox': det['bbox'],
            'bbox_normalized': det['bbox_normalized'],
            'model': 'oi',
        })

for imgid in det_results_coco_seed:
    for det in det_results_coco_seed[imgid]:
        label = det['label']
        score = det['score']
            
        transformed_score = (((score - norm_coef_coco[label]['mean_coco']) 
                              * norm_coef_coco[label]['std_oi'] / norm_coef_coco[label]['std_coco'] 
                              + norm_coef_coco[label]['mean_oi']) 
                             if label in norm_coef_coco else (
                                 ((score - norm_coef_coco['default']['mean_coco']) 
                                  * norm_coef_coco['default']['std_oi'] / norm_coef_coco['default']['std_coco'] 
                                  + norm_coef_coco['default']['mean_oi']) 
                             ))
        transformed_score = min(max(transformed_score, 0.), 1.)
                
        det_results_concat[imgid].append({
            'label': label,
            'score': transformed_score,
            'bbox': det['bbox'],
            'bbox_normalized': det['bbox_normalized'],
            'model': 'coco',
        })

for imgid in det_results_pascal_seed:
    for det in det_results_pascal_seed[imgid]:
        label = det['label']
        score = det['score']
            
        transformed_score = (((score - norm_coef_pascal[label]['mean_pascal']) 
                              * norm_coef_pascal[label]['std_oi'] / norm_coef_pascal[label]['std_pascal'] 
                              + norm_coef_pascal[label]['mean_oi']) 
                             if label in norm_coef_pascal else (
                                 ((score - norm_coef_pascal['default']['mean_pascal']) 
                                  * norm_coef_pascal['default']['std_oi'] / norm_coef_pascal['default']['std_pascal'] 
                                  + norm_coef_pascal['default']['mean_oi']) 
                             ))
        transformed_score = min(max(transformed_score, 0.), 1.)
                
        det_results_concat[imgid].append({
            'label': label,
            'score': transformed_score,
            'bbox': det['bbox'],
            'bbox_normalized': det['bbox_normalized'],
            'model': 'pascal',
        })
                
for imgid in det_results_ws_seed:
    for det in det_results_ws_seed[imgid]:
        label = det['label']
        score = det['score']
            
        transformed_score = (((score - norm_coef_ws[label]['mean_ws']) 
                              * norm_coef_ws[label]['std_oi'] / norm_coef_ws[label]['std_ws'] 
                              + norm_coef_ws[label]['mean_oi']) 
                             if label in norm_coef_ws else (
                                 ((score - norm_coef_ws['default']['mean_ws']) 
                                  * norm_coef_ws['default']['std_oi'] / norm_coef_ws['default']['std_ws'] 
                                  + norm_coef_ws['default']['mean_oi']) 
                             ))
        transformed_score = min(max(transformed_score, 0.), 1.)
                
        det_results_concat[imgid].append({
            'label': label,
            'score': transformed_score,
            'bbox': det['bbox'],
            'bbox_normalized': det['bbox_normalized'],
            'model': 'ws',
        })
        

In [10]:
with open('../../results/det_results_concat_25.pkl', 'wb') as fout:
    pickle.dump(det_results_concat, fout)

## Merging duplicate results

In [12]:
def iou(det_bbox, gt_bbox):
    x_d_len = det_bbox[2] - det_bbox[0]
    y_d_len = det_bbox[3] - det_bbox[1]
    x_t_len = gt_bbox[2] - gt_bbox[0]
    y_t_len = gt_bbox[3] - gt_bbox[1]
    x_int_len = max(0, min(gt_bbox[2], det_bbox[2]) - max(gt_bbox[0], det_bbox[0]))
    y_int_len = max(0, min(gt_bbox[3], det_bbox[3]) - max(gt_bbox[1], det_bbox[1]))
    iou = (x_int_len*y_int_len) / (x_d_len*y_d_len + x_t_len*y_t_len - x_int_len*y_int_len)
    return iou
'''
def ioa(det_bbox, gt_bbox):
    x_d_len = det_bbox[2] - det_bbox[0]
    y_d_len = det_bbox[3] - det_bbox[1]
    x_t_len = gt_bbox[2] - gt_bbox[0]
    y_t_len = gt_bbox[3] - gt_bbox[1]
    x_int_len = max(0, min(gt_bbox[2], det_bbox[2]) - max(gt_bbox[0], det_bbox[0]))
    y_int_len = max(0, min(gt_bbox[3], det_bbox[3]) - max(gt_bbox[1], det_bbox[1]))
    iou = (x_int_len*y_int_len) / (x_d_len*y_d_len)
    return iou
'''

'\ndef ioa(det_bbox, gt_bbox):\n    x_d_len = det_bbox[2] - det_bbox[0]\n    y_d_len = det_bbox[3] - det_bbox[1]\n    x_t_len = gt_bbox[2] - gt_bbox[0]\n    y_t_len = gt_bbox[3] - gt_bbox[1]\n    x_int_len = max(0, min(gt_bbox[2], det_bbox[2]) - max(gt_bbox[0], det_bbox[0]))\n    y_int_len = max(0, min(gt_bbox[3], det_bbox[3]) - max(gt_bbox[1], det_bbox[1]))\n    iou = (x_int_len*y_int_len) / (x_d_len*y_d_len)\n    return iou\n'

In [14]:
_STAT_num_same_merged = 0
_STAT_num_diff_merged = 0

thresh_same = 0.5
thresh_diff = 0.7

det_results_merged = {}
for imgid, det in det_results_concat.items():
    det_results_merged[imgid] = []
    groups = []
    for ii in range(len(det)):
        matching_gr = None
        for gr in groups:
            for item in gr:
                if det[ii]['label'] == det[item]['label'] and iou(det[ii]['bbox'], det[item]['bbox']) > thresh_same:
                    if matching_gr == None:
                        gr.append(ii)
                        matching_gr = gr
                    else:
                        matching_gr += gr
                        gr.clear()
                    _STAT_num_same_merged += 1
                    break
                if det[ii]['label'] != det[item]['label'] and iou(det[ii]['bbox'], det[item]['bbox']) > thresh_diff:
                    if matching_gr == None:
                        gr.append(ii)
                        matching_gr = gr
                    else:
                        matching_gr += gr
                        gr.clear()
                    _STAT_num_diff_merged += 1
                    break
                
        if matching_gr == None:
            groups.append([ii])
            
    for g in groups:
        if len(g) == 0:
            continue
        suff = '/J' if len(g) > 1 else ''
        scores = [det[ii]['score'] if det[ii]['model'] != 'ws' else 0. for ii in g]
        iii = np.argmax(scores)
        det_results_merged[imgid].append({
            'label': det[g[iii]]['label'],
            'score': det[g[iii]]['score'],
            'bbox': det[g[iii]]['bbox'],
            'bbox_normalized': det[g[iii]]['bbox_normalized'],
            'model': det[g[iii]]['model'] + suff,            
        })

In [16]:
_STAT_num_same_merged, _STAT_num_diff_merged

(344087, 920608)

In [15]:
with open('../../results/det_results_merged_25.pkl', 'wb') as fout:
    pickle.dump(det_results_merged, fout)